# జనరేటివ్ నెట్‌వర్క్స్

రీకరెంట్ న్యూరల్ నెట్‌వర్క్స్ (RNNs) మరియు వాటి గేటెడ్ సెల్ వేరియంట్లు, ఉదాహరణకు లాంగ్ షార్ట్ టర్మ్ మెమరీ సెల్స్ (LSTMs) మరియు గేటెడ్ రీకరెంట్ యూనిట్స్ (GRUs), భాషా మోడలింగ్ కోసం ఒక యంత్రాంగాన్ని అందించాయి, అంటే అవి పదాల క్రమాన్ని నేర్చుకుని, ఒక సీక్వెన్స్‌లో తదుపరి పదం కోసం అంచనాలు ఇవ్వగలవు. ఇది RNNలను **జనరేటివ్ పనుల** కోసం ఉపయోగించడానికి అనుమతిస్తుంది, ఉదాహరణకు సాధారణ టెక్స్ట్ జనరేషన్, మెషీన్ అనువాదం, మరియు ఇమేజ్ క్యాప్షనింగ్ కూడా.

మునుపటి యూనిట్‌లో చర్చించిన RNN నిర్మాణంలో, ప్రతి RNN యూనిట్ తదుపరి హిడెన్ స్టేట్‌ను అవుట్‌పుట్‌గా ఉత్పత్తి చేసింది. అయితే, ప్రతి రీకరెంట్ యూనిట్‌కు మరో అవుట్‌పుట్‌ను కూడా జోడించవచ్చు, ఇది మాకు ఒక **సీక్వెన్స్** (మూల సీక్వెన్స్ పొడవుతో సమానం) అవుట్‌పుట్ ఇవ్వడానికి అనుమతిస్తుంది. అంతేకాకుండా, ప్రతి దశలో ఇన్‌పుట్‌ను స్వీకరించని RNN యూనిట్లను ఉపయోగించవచ్చు, అవి కొంత ప్రారంభ స్థితి వెక్టర్‌ను మాత్రమే తీసుకుని, ఆపై అవుట్‌పుట్‌ల సీక్వెన్స్‌ను ఉత్పత్తి చేస్తాయి.

ఈ నోట్‌బుక్‌లో, మేము టెక్స్ట్‌ను ఉత్పత్తి చేయడంలో సహాయపడే సాదా జనరేటివ్ మోడల్స్‌పై దృష్టి సారిస్తాము. సౌలభ్యానికి, మనం **అక్షర స్థాయి నెట్‌వర్క్**ను నిర్మిద్దాం, ఇది అక్షరాల వారీగా టెక్స్ట్‌ను ఉత్పత్తి చేస్తుంది. శిక్షణ సమయంలో, మనం కొంత టెక్స్ట్ కార్పస్ తీసుకుని, దాన్ని అక్షరాల సీక్వెన్స్‌లుగా విభజించాలి.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## అక్షర పదసంపద నిర్మాణం

అక్షర స్థాయి జనరేటివ్ నెట్‌వర్క్‌ను నిర్మించడానికి, మేము వచనం పదాల స్థానంలో వ్యక్తిగత అక్షరాలుగా విభజించాలి. ఇది వేరే టోకనైజర్‌ను నిర్వచించడం ద్వారా చేయవచ్చు:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


మన డేటాసెట్ నుండి టెక్స్ట్‌ను ఎలా ఎన్‌కోడ్ చేయగలమో ఉదాహరణ చూద్దాం:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## జనరేటివ్ RNN శిక్షణ

RNN ను టెక్స్ట్ ఉత్పత్తి చేయడానికి శిక్షణ ఇవ్వడానికి మనం అనుసరించే విధానం ఈ విధంగా ఉంటుంది. ప్రతి దశలో, మనం `nchars` పొడవైన అక్షరాల సీక్వెన్స్ తీసుకుని, ప్రతి ఇన్‌పుట్ అక్షరానికి తదుపరి అవుట్‌పుట్ అక్షరాన్ని నెట్‌వర్క్ ఉత్పత్తి చేయమని అడుగుతాము:

!['HELLO' అనే పదం RNN ద్వారా ఉత్పత్తి చేయబడుతున్న ఉదాహరణను చూపించే చిత్రం.](../../../../../translated_images/te/rnn-generate.56c54afb52f9781d.webp)

నిజమైన పరిస్థితులపై ఆధారపడి, మనం కొన్ని ప్రత్యేక అక్షరాలను కూడా చేర్చాలనుకోవచ్చు, ఉదాహరణకు *end-of-sequence* `<eos>`. మన సందర్భంలో, మనం నిరంతర టెక్స్ట్ ఉత్పత్తి కోసం నెట్‌వర్క్‌ను శిక్షణ ఇవ్వాలనుకుంటున్నాము, కాబట్టి ప్రతి సీక్వెన్స్ పరిమాణాన్ని `nchars` టోకెన్లకు సమానంగా స్థిరపరుస్తాము. ఫలితంగా, ప్రతి శిక్షణ ఉదాహరణలో `nchars` ఇన్‌పుట్లు మరియు `nchars` అవుట్‌పుట్లు ఉంటాయి (ఇవి ఇన్‌పుట్ సీక్వెన్స్‌ను ఒక అక్షరం ఎడమకు షిఫ్ట్ చేసినవి). మినీబ్యాచ్‌లో ఇలాంటి అనేక సీక్వెన్స్‌లు ఉంటాయి.

మినీబ్యాచ్‌లను ఉత్పత్తి చేసే విధానం ఏమిటంటే, ప్రతి వార్తా టెక్స్ట్ `l` పొడవు ఉన్నదాన్ని తీసుకుని, అందులోని అన్ని సాధ్యమైన ఇన్‌పుట్-అవుట్‌పుట్ కలయికలను ఉత్పత్తి చేయడం (ఇవి `l-nchars` సంఖ్యలో ఉంటాయి). అవి ఒక మినీబ్యాచ్‌ను ఏర్పరుస్తాయి, మరియు శిక్షణ ప్రతి దశలో మినీబ్యాచ్ పరిమాణం వేరుగా ఉంటుంది.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

ఇప్పుడు జనరేటర్ నెట్‌వర్క్‌ను నిర్వచిద్దాం. ఇది మునుపటి యూనిట్‌లో చర్చించిన ఏదైనా రికరెంట్ సెల్ ఆధారంగా ఉండవచ్చు (సింపుల్, LSTM లేదా GRU). మన ఉదాహరణలో మనం LSTM ఉపయోగిస్తాము.

నెట్‌వర్క్ అక్షరాలను ఇన్‌పుట్‌గా తీసుకుంటుంది, మరియు పదసంపద పరిమాణం చాలా చిన్నది కాబట్టి, ఎంబెడ్డింగ్ లేయర్ అవసరం లేదు, వన్-హాట్-ఎంకోడ్ చేసిన ఇన్‌పుట్ నేరుగా LSTM సెల్‌కు వెళ్లవచ్చు. అయితే, మనం అక్షర సంఖ్యలను ఇన్‌పుట్‌గా పంపిస్తున్నందున, వాటిని LSTM‌కు పంపించే ముందు వన్-హాట్-ఎంకోడ్ చేయాలి. ఇది `forward` పాస్ సమయంలో `one_hot` ఫంక్షన్‌ను పిలవడం ద్వారా జరుగుతుంది. అవుట్‌పుట్ ఎంకోడర్ ఒక లీనియర్ లేయర్ అవుతుంది, ఇది హిడెన్ స్టేట్‌ను వన్-హాట్-ఎంకోడ్ చేసిన అవుట్‌పుట్‌గా మార్చుతుంది.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

శిక్షణ సమయంలో, మేము ఉత్పత్తి చేసిన టెక్స్ట్‌ను నమూనా చేయగలగాలి. దానికి, మేము `generate` అనే ఫంక్షన్‌ను నిర్వచిస్తాము, ఇది ప్రారంభ స్ట్రింగ్ `start` నుండి ప్రారంభించి `size` పొడవైన అవుట్పుట్ స్ట్రింగ్‌ను ఉత్పత్తి చేస్తుంది.

ఇది పనిచేసే విధానం ఇలా ఉంటుంది. మొదట, మేము మొత్తం ప్రారంభ స్ట్రింగ్‌ను నెట్‌వర్క్ ద్వారా పంపించి, అవుట్పుట్ స్టేట్ `s` మరియు తదుపరి అంచనా వేసిన అక్షరం `out` ను తీసుకుంటాము. `out` ఒక హాట్ ఎన్‌కోడ్ చేయబడినది కాబట్టి, అక్షర సూచిక `nc` ను పొందడానికి `argmax` తీసుకుంటాము, మరియు `itos` ఉపయోగించి నిజమైన అక్షరాన్ని తెలుసుకుని, దాన్ని ఫలితంగా వచ్చిన అక్షరాల జాబితా `chars` కు జోడిస్తాము. ఒక అక్షరం ఉత్పత్తి చేసే ఈ ప్రక్రియను అవసరమైన అక్షరాల సంఖ్య `size` సార్లు పునరావృతం చేస్తాము.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

ఇప్పుడు ట్రైనింగ్ చేద్దాం! ట్రైనింగ్ లూప్ మన గత ఉదాహరణలన్నింటిలో ఉన్నట్లే ఉంటుంది, కానీ ఖచ్చితత్వం బదులు ప్రతి 1000 epochsకి ఒక నమూనా రూపొందించిన టెక్స్ట్ ముద్రిస్తాము.

నష్టాన్ని లెక్కించే విధానంపై ప్రత్యేక శ్రద్ధ అవసరం. మనం ఒక హాట్-ఎంకోడెడ్ అవుట్‌పుట్ `out` మరియు అంచనా టెక్స్ట్ `text_out` (అక్షర సూచికల జాబితా) ఇవ్వబడినప్పుడు నష్టాన్ని లెక్కించాలి. అదృష్టవశాత్తు, `cross_entropy` ఫంక్షన్ మొదటి ఆర్గ్యుమెంట్‌గా సాధారణీకరించని నెట్‌వర్క్ అవుట్‌పుట్‌ను, రెండవ ఆర్గ్యుమెంట్‌గా క్లాస్ నంబర్‌ను ఆశిస్తుంది, ఇది మన దగ్గర ఉన్నదే. ఇది మినీబ్యాచ్ పరిమాణంపై ఆటోమేటిక్ సగటు కూడా చేస్తుంది.

మేము ట్రైనింగ్‌ను `samples_to_train` నమూనాల వరకు పరిమితం చేస్తాము, చాలా ఎక్కువ సమయం వేచి ఉండకుండా ఉండేందుకు. మీరు ప్రయోగాలు చేసి, ఎక్కువ epochs కోసం (అప్పుడు ఈ కోడ్ చుట్టూ మరో లూప్ సృష్టించాల్సి ఉంటుంది) దీర్ఘకాలిక ట్రైనింగ్ ప్రయత్నించమని మేము ప్రోత్సహిస్తాము.


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

ఈ ఉదాహరణ ఇప్పటికే మంచి టెక్స్ట్‌ను ఉత్పత్తి చేస్తోంది, కానీ దీన్ని మరింత మెరుగుపరచవచ్చు కొన్ని మార్గాల్లో:
* **మినీబ్యాచ్ ఉత్పత్తి మెరుగుపరచడం**. ట్రైనింగ్ కోసం డేటాను సిద్ధం చేసిన విధానం ఒక్కో నమూనా నుండి ఒక మినీబ్యాచ్‌ను ఉత్పత్తి చేయడం. ఇది సరైనది కాదు, ఎందుకంటే మినీబ్యాచ్లు అన్ని వేర్వేరు పరిమాణాలుగా ఉంటాయి, మరియు వాటిలో కొన్ని ఉత్పత్తి చేయలేవు, ఎందుకంటే టెక్స్ట్ `nchars` కంటే చిన్నది. అలాగే, చిన్న మినీబ్యాచ్లు GPU ని తగినంత లోడ్ చేయవు. అందుకే అన్ని నమూనాల నుండి ఒక పెద్ద టెక్స్ట్ భాగాన్ని తీసుకుని, అన్ని ఇన్‌పుట్-అవుట్‌పుట్ జంటలను ఉత్పత్తి చేసి, వాటిని షఫుల్ చేసి, సమాన పరిమాణం గల మినీబ్యాచ్లు ఉత్పత్తి చేయడం మంచిది.
* **బహుళస్థాయి LSTM**. 2 లేదా 3 LSTM లేయర్లను ప్రయత్నించడం అర్థం. మునుపటి యూనిట్‌లో చెప్పినట్లుగా, ప్రతి LSTM లేయర్ టెక్స్ట్ నుండి నిర్దిష్ట నమూనాలను తీసుకుంటుంది, మరియు క్యారెక్టర్-స్థాయి జనరేటర్ సందర్భంలో తక్కువ LSTM స్థాయి సిలబుల్స్‌ను తీసుకోవడానికి బాధ్యత వహిస్తుంది, మరియు ఉన్నత స్థాయిలు పదాలు మరియు పద సంయోజనాల కోసం. ఇది LSTM కన్‌స్ట్రక్టర్‌కు లేయర్ల సంఖ్య పారామీటర్‌ను అందించడం ద్వారా సులభంగా అమలు చేయవచ్చు.
* మీరు **GRU యూనిట్లతో** కూడా ప్రయోగాలు చేయవచ్చు మరియు ఏవి మెరుగ్గా పనిచేస్తాయో చూడవచ్చు, అలాగే **విభిన్న హిడెన్ లేయర్ పరిమాణాలతో** కూడా. చాలా పెద్ద హిడెన్ లేయర్ ఓవర్‌ఫిట్టింగ్‌కు దారితీయవచ్చు (ఉదా: నెట్‌వర్క్ ఖచ్చితమైన టెక్స్ట్ నేర్చుకుంటుంది), మరియు చిన్న పరిమాణం మంచి ఫలితాన్ని ఇవ్వకపోవచ్చు.


## సాఫ్ట్ టెక్స్ట్ జనరేషన్ మరియు టెంపరేచర్

మునుపటి `generate` నిర్వచనంలో, మేము ఎప్పుడూ అత్యధిక సంభావ్యత కలిగిన అక్షరాన్ని తదుపరి అక్షరంగా తీసుకుంటున్నాము. దీని ఫలితంగా, టెక్స్ట్ తరచుగా అదే అక్షర శ్రేణుల మధ్య "సైకిల్" అవుతుంది, ఈ ఉదాహరణలో కనిపించేదిలా:
```
today of the second the company and a second the company ...
```

కానీ, తదుపరి అక్షరానికి సంభావ్యత పంపిణీని చూస్తే, కొన్ని అత్యధిక సంభావ్యతల మధ్య తేడా పెద్దది కాకపోవచ్చు, ఉదాహరణకు ఒక అక్షరానికి 0.2 సంభావ్యత ఉండవచ్చు, మరొకదానికి 0.19, మొదలైనవి. ఉదాహరణకు, '*play*' శ్రేణిలో తదుపరి అక్షరం కోసం చూస్తున్నప్పుడు, అది స్పేస్ లేదా **e** (పదం *player* లో ఉన్నట్లు) రెండూ సమానంగా ఉండవచ్చు.

దీని ద్వారా మనం తెలుసుకోవచ్చు, ఎప్పుడూ ఎక్కువ సంభావ్యత కలిగిన అక్షరాన్ని ఎంచుకోవడం "న్యాయం" కాదు, ఎందుకంటే రెండవ అత్యధికాన్ని ఎంచుకున్నా అర్థవంతమైన టెక్స్ట్ రావచ్చు. నెట్‌వర్క్ అవుట్‌పుట్ ఇచ్చిన సంభావ్యత పంపిణీ నుండి అక్షరాలను **సాంపిల్** చేయడం మేలు.

ఈ సాంప్లింగ్ `multinomial` ఫంక్షన్ ఉపయోగించి చేయవచ్చు, ఇది **మల్టినోమియల్ పంపిణీ**ని అమలు చేస్తుంది. ఈ **సాఫ్ట్** టెక్స్ట్ జనరేషన్‌ను అమలు చేసే ఫంక్షన్ క్రింద నిర్వచించబడింది:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

మేము **తాపన** అనే మరో పారామితిని పరిచయం చేసాము, ఇది మనం అత్యధిక సంభావ్యతకు ఎంత కట్టుబడి ఉండాలో సూచించడానికి ఉపయోగిస్తారు. తాపన 1.0 అయితే, మనం సరాసరి మల్టినోమియల్ శాంప్లింగ్ చేస్తాము, మరియు తాపన అనంతానికి చేరినప్పుడు - అన్ని సంభావ్యతలు సమానంగా మారతాయి, మరియు మనం యాదృచ్ఛికంగా తదుపరి అక్షరాన్ని ఎంచుకుంటాము. క్రింది ఉదాహరణలో, తాపనను చాలా పెంచినప్పుడు టెక్స్ట్ అర్థరహితంగా మారిపోతుందని, మరియు అది 0కి దగ్గరగా ఉన్నప్పుడు "సైకిల్డ్" కఠినంగా ఉత్పత్తి చేసిన టెక్స్ట్ లాగా కనిపిస్తుందని మనం గమనించవచ్చు.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**అస్పష్టత**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించబడింది. మేము ఖచ్చితత్వానికి ప్రయత్నించినప్పటికీ, ఆటోమేటెడ్ అనువాదాల్లో పొరపాట్లు లేదా తప్పిదాలు ఉండవచ్చు. మూల పత్రం దాని స్వదేశీ భాషలోనే అధికారిక మూలంగా పరిగణించాలి. ముఖ్యమైన సమాచారానికి, ప్రొఫెషనల్ మానవ అనువాదం చేయించుకోవడం మంచిది. ఈ అనువాదం వలన కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారుల బాధ్యత మేము తీసుకోము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
